In [32]:
import re
import warnings

import lightgbm as lgb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import spacy
from bert_logistic import read_texts_from_dir
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import word_tokenize, sent_tokenize


warnings.filterwarnings("ignore", category=UserWarning)

In [33]:
try:
    nltk.data.find("tokenizers/punkt")
except nltk.downloader.DownloadError:
    nltk.download("punkt")

# Tải mô hình spacy
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Mô hình 'en_core_web_sm' chưa được tải. Vui lòng chạy:")
    print("python -m spacy download en_core_web_sm")
    nlp = None

In [34]:
train_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train"
test_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test"
gt_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train.csv"
print("Loading data...")
df_train = read_texts_from_dir(train_path)
df_test = read_texts_from_dir(test_path)
df_train_gt = pd.read_csv(gt_path)
y_train = df_train_gt["real_text_id"].values

df_train.rename(columns={"file_1": "text_0", "file_2": "text_1"}, inplace=True)
df_test.rename(columns={"file_1": "text_0", "file_2": "text_1"}, inplace=True)
df_train["label"] = df_train_gt["real_text_id"]

df_train["label"] = df_train["label"].map({1: 0, 2: 1})

Loading data...
Number of directories: 95
Number of directories: 1068


In [35]:
df_train['word_count_text_0'] = df_train['text_0'].apply(lambda x: len(word_tokenize(x)))
df_train['word_count_text_1'] = df_train['text_1'].apply(lambda x: len(word_tokenize(x)))

df_train['sentence_count_text_0'] = df_train['text_0'].apply(lambda x: len(sent_tokenize(x)))
df_train['sentence_count_text_1'] = df_train['text_1'].apply(lambda x: len(sent_tokenize(x)))

df_train['ner_count_text_0'] = df_train['text_0'].apply(lambda x: len(nlp(x).ents))
df_train['ner_count_text_1'] = df_train['text_1'].apply(lambda x: len(nlp(x).ents))

In [36]:
# Các ngưỡng bạn đã chọn (hoặc từ Decision Tree)
THRESHOLDS = {
    'word_count_max': 550,
    'word_count_min': 80,
    'sentence_count_max': 25,
    'sentence_count_min': 2,
    'ner_count_max': 58
}

def is_abnormal(word_count, sentence_count, ner_count, thresholds):
    """Kiểm tra một văn bản có đặc điểm bất thường không. Trả về True/False."""
    if word_count > thresholds['word_count_max'] or word_count < thresholds['word_count_min']:
        return True
    if sentence_count > thresholds['sentence_count_max'] or sentence_count < thresholds['sentence_count_min']:
        return True
    if ner_count > thresholds['ner_count_max']:
        return True
    return False


def hybrid_classifier_logic(row, thresholds):
    """
    Áp dụng logic lai cho một hàng dữ liệu.
    - Trả về 0 hoặc 1 nếu quy tắc có thể quyết định.
    - Trả về -1 (hoặc None) nếu là trường hợp khó, cần mô hình ML.
    """
    # Trích xuất các feature đã được tính toán sẵn trong DataFrame của bạn
    # (Giả sử bạn đã chạy hàm create_numerical_features và có các cột này)
    features_0 = (row.word_count_text_0, row.sentence_count_text_0, row.ner_count_text_0)
    features_1 = (row.word_count_text_1, row.sentence_count_text_1, row.ner_count_text_1)

    # Áp dụng bộ lọc
    is_abnormal_0 = is_abnormal(*features_0, thresholds)
    is_abnormal_1 = is_abnormal(*features_1, thresholds)

    # Áp dụng logic
    if is_abnormal_0 and not is_abnormal_1:
        # text_0 là FAKE, text_1 là REAL -> dự đoán nhãn là 1
        return 1
    elif not is_abnormal_0 and is_abnormal_1:
        # text_0 là REAL, text_1 là FAKE -> dự đoán nhãn là 0
        return 0
    else:
        # Cả hai đều bình thường, HOẶC cả hai đều bất thường.
        # Đây là trường hợp "khó", cần mô hình ML xử lý.
        return -1 # Dùng -1 để đánh dấu


In [37]:
df_train['initial_prediction'] = df_train.apply(lambda row: hybrid_classifier_logic(row, THRESHOLDS), axis=1)

ambiguous_indices = df_train[df_train['initial_prediction'] == -1].index

df_model = df_train.loc[ambiguous_indices]
print(df_model.shape)

(66, 10)


In [38]:
df_train[df_train['initial_prediction'] != -1][['label', 'initial_prediction']]

,label,initial_prediction
id,,
4,1,1
10,0,0
12,0,0
14,1,1
16,0,0
34,1,1
40,0,0
45,1,1
55,1,1


In [39]:
df_model = df_model[['text_0', 'text_1', 'label']]
y = df_model['label']
df_model.rename(columns={"text_0": "file_1", "text_1": "file_2"}, inplace=True)
df_model.head()

,file_1,file_2,label
id,,,
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...,0
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...,1
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...,0
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...,1
5,"Since its launch in '99, the Very Large Telesc...",AssemblyCulture AssemblyCulture AssemblyCultur...,0


In [40]:
from bert_logistic import prepare_data_for_model

In [41]:
X, embedding_extractor = prepare_data_for_model(
    df_model, 
    fit_embedding=True, 
    model_name='intfloat/multilingual-e5-small'
)

Step 1: Extracting top importance features...


Extracting top features: 100%|██████████| 66/66 [00:00<00:00, 393.22it/s]


Step 2: Extracting rule-based features...


Extracting rule-based features: 100%|██████████| 66/66 [00:00<00:00, 242.23it/s]


Step 3: Extracting statistical features...
Step 4: Extracting embedding features...
Loading embedding model: intfloat/multilingual-e5-small
Loaded as SentenceTransformer model
Extracting embedding features for file_1...


...

Extracting embedding features for file_2...


Extracting embeddings: 100%|██████████| 2/2 [00:03<00:00,  1.78s/it]


Step 5: Extracting pairwise features...


Extracting pairwise features: 100%|██████████| 66/66 [00:00<00:00, 547.31it/s]

Step 6: Combining features...
Final feature matrix shape: (66, 1654)
Top features: 25, Rule: 78, Stat: 6, Embedding: 1538, Pairwise: 7


In [45]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0)

# Train the model
catboost_model.fit(X_train, y_train)

# Predict on validation set
y_preds_val_catboost = catboost_model.predict(X_val)

# Calculate accuracy
acc_catboost = accuracy_score(y_val, y_preds_val_catboost)
print(f"Validation accuracy with CatBoost: {acc_catboost:.4f}")

Validation accuracy with CatBoost: 0.9286


In [50]:
import pandas as pd
import numpy as np
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from catboost import CatBoostClassifier

# Giả sử các file và hàm custom của bạn đã có sẵn
from bert_logistic import read_texts_from_dir, prepare_data_for_model

# === BƯỚC 1: TẢI DỮ LIỆU VÀ CHUẨN BỊ BAN ĐẦU ===

print("--- BƯỚC 1: Tải và chuẩn bị dữ liệu ---")
# Tải dữ liệu train và test
train_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train"
test_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test"
gt_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train.csv"

df_train = read_texts_from_dir(train_path)
df_test = read_texts_from_dir(test_path)
df_train_gt = pd.read_csv(gt_path)

# Gán nhãn và đổi tên cột
df_train.rename(columns={"file_1": "text_0", "file_2": "text_1"}, inplace=True)
df_test.rename(columns={"file_1": "text_0", "file_2": "text_1"}, inplace=True)
df_train["label"] = df_train_gt["real_text_id"].map({1: 0, 2: 1})

# Tải mô hình spacy
nlp = spacy.load("en_core_web_sm")
print("✅ Dữ liệu đã sẵn sàng.")

# === BƯỚC 2 (ĐÃ CẬP NHẬT): ÁP DỤNG BỘ LỌC RULE-BASED VỚI LOGIC RÕ RÀNG HƠN ===

print("\n--- BƯỚC 2: Áp dụng bộ lọc Rule-Based cho tập Train ---")
# Tính các feature cơ bản cho việc lọc (giữ nguyên)
df_train['word_count_text_0'] = df_train['text_0'].apply(lambda x: len(word_tokenize(x)))
df_train['word_count_text_1'] = df_train['text_1'].apply(lambda x: len(word_tokenize(x)))
df_train['sentence_count_text_0'] = df_train['text_0'].apply(lambda x: len(sent_tokenize(x)))
df_train['sentence_count_text_1'] = df_train['text_1'].apply(lambda x: len(sent_tokenize(x)))
df_train['ner_count_text_0'] = df_train['text_0'].apply(lambda x: len(nlp(x).ents))
df_train['ner_count_text_1'] = df_train['text_1'].apply(lambda x: len(nlp(x).ents))

# Định nghĩa lại các hàm lọc (giữ nguyên)
THRESHOLDS = {'word_count_max': 550, 'word_count_min': 80, 'sentence_count_max': 25, 'sentence_count_min': 2, 'ner_count_max': 58}
def is_abnormal(wc, sc, nc, t):
    if wc > t['word_count_max'] or wc < t['word_count_min']: return True
    if sc > t['sentence_count_max'] or sc < t['sentence_count_min']: return True
    if nc > t['ner_count_max']: return True
    return False

# ----- THAY THẾ BẰNG PHIÊN BẢN LOGIC V2 RÕ RÀNG HƠN -----
def hybrid_classifier_logic_v2(row, thresholds):
    """
    Phiên bản logic lai rõ ràng hơn để thể hiện tính đối xứng.
    """
    features_0 = (row.word_count_text_0, row.sentence_count_text_0, row.ner_count_text_0)
    features_1 = (row.word_count_text_1, row.sentence_count_text_1, row.ner_count_text_1)

    is_abnormal_0 = is_abnormal(*features_0, thresholds)
    is_abnormal_1 = is_abnormal(*features_1, thresholds)

    # TRƯỜNG HỢP 1: Quy tắc có thể quyết định rõ ràng (chỉ một bên bất thường)
    if is_abnormal_0 != is_abnormal_1:
        if is_abnormal_0:
            # text_0 bất thường (FAKE), vậy text_1 là REAL -> dự đoán 1
            return 1
        else: # is_abnormal_1 là True
            # text_1 bất thường (FAKE), vậy text_0 là REAL -> dự đoán 0
            return 0
            
    # TRƯỜNG HỢP 2: Quy tắc không thể quyết định (cả hai cùng bình thường hoặc cùng bất thường)
    else:
        return -1 # Cần mô hình ML
# ----- KẾT THÚC PHẦN THAY THẾ -----

# Tách ra các mẫu "khó" để huấn luyện mô hình ML
# Sử dụng hàm logic v2 mới
df_train['initial_prediction'] = df_train.apply(lambda row: hybrid_classifier_logic_v2(row, THRESHOLDS), axis=1)
ambiguous_train_indices = df_train[df_train['initial_prediction'] == -1].index
df_model_train = df_train.loc[ambiguous_train_indices][['text_0', 'text_1', 'label']]
df_model_train.rename(columns={"text_0": "file_1", "text_1": "file_2"}, inplace=True)
y_model_train = df_model_train['label']

print(f"✅ Đã tách được {len(df_model_train)} mẫu 'khó' từ tập train để huấn luyện model.")

# === BƯÓC 3: HUẤN LUYỆN MODEL CUỐI CÙNG ===

print("\n--- BƯỚC 3: Huấn luyện mô hình CatBoost cuối cùng ---")
# Chuẩn bị dữ liệu train cho model bằng hàm của bạn
# 'fit_embedding=True' để huấn luyện extractor trên dữ liệu này
X_model_train, embedding_extractor = prepare_data_for_model(
    df_model_train,
    fit_embedding=True,
    model_name='intfloat/multilingual-e5-small'
)

# Huấn luyện model trên toàn bộ 66 mẫu "khó"
final_catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0, random_state=42)
final_catboost_model.fit(X_model_train, y_model_train)
print("✅ Huấn luyện mô hình CatBoost cuối cùng hoàn tất.")

# === BƯỚC 4 (ĐÃ CẬP NHẬT): ÁP DỤNG HYBRID APPROACH CHO TẬP TEST ===

print("\n--- BƯỚC 4: Áp dụng Hybrid Approach cho tập Test ---")
# Ensure all required features are computed for df_test
df_test['sentence_count_text_0'] = df_test['text_0'].apply(lambda x: len(sent_tokenize(x)))
df_test['sentence_count_text_1'] = df_test['text_1'].apply(lambda x: len(sent_tokenize(x)))
df_test['word_count_text_1'] = df_test['text_1'].apply(lambda x: len(word_tokenize(x)))
df_test['word_count_text_0'] = df_test['text_0'].apply(lambda x: len(word_tokenize(x)))
df_test['ner_count_text_0'] = df_test['text_0'].apply(lambda x: len(nlp(x).ents))
df_test['ner_count_text_1'] = df_test['text_1'].apply(lambda x: len(nlp(x).ents))

# 4b. Áp dụng bộ lọc rule-based (sử dụng hàm v2)
df_test['prediction'] = df_test.apply(lambda row: hybrid_classifier_logic_v2(row, THRESHOLDS), axis=1)
ambiguous_test_indices = df_test[df_test['prediction'] == -1].index
df_model_test = df_test.loc[ambiguous_test_indices][['text_0', 'text_1']]
df_model_test.rename(columns={"text_0": "file_1", "text_1": "file_2"}, inplace=True)


print(f"Bộ lọc Rule-based đã xử lý {len(df_test) - len(df_model_test)} mẫu 'dễ' trên tập test.")
print(f"Còn lại {len(df_model_test)} mẫu 'khó' cần mô hình ML xử lý.")

# 4c. Dùng model ML cho các trường hợp "khó"
if not df_model_test.empty:
    # Chuẩn bị dữ liệu test cho model
    # 'fit_embedding=False' và truyền extractor đã được huấn luyện
    X_model_test, _ = prepare_data_for_model(
        df_model_test,
        fit_embedding=False,
        embedding_extractor=embedding_extractor
    )
    
    # Dự đoán
    ml_predictions = final_catboost_model.predict(X_model_test)
    
    # Cập nhật kết quả dự đoán
    df_test.loc[ambiguous_test_indices, 'prediction'] = ml_predictions
print("✅ Dự đoán cho các mẫu 'khó' hoàn tất.")


# === BƯỚC 5: TẠO FILE SUBMISSION ===
print("\n--- BƯỚC 5: Tạo file submission ---")
submission_df = pd.DataFrame()
submission_df['id'] = df_test.index
submission_df['real_text_id'] = df_test['prediction'].astype(int)

# Chuyển nhãn từ (0, 1) trở lại (1, 2)
submission_df['real_text_id'] = submission_df['real_text_id'].map({0: 1, 1: 2})

submission_df.to_csv('submission.csv', index=False)
print("✅ Đã tạo file submission.csv thành công!")
print(submission_df.head())

--- BƯỚC 1: Tải và chuẩn bị dữ liệu ---
Number of directories: 95
Number of directories: 1068
✅ Dữ liệu đã sẵn sàng.

--- BƯỚC 2: Áp dụng bộ lọc Rule-Based cho tập Train ---
✅ Đã tách được 66 mẫu 'khó' từ tập train để huấn luyện model.

--- BƯỚC 3: Huấn luyện mô hình CatBoost cuối cùng ---
Step 1: Extracting top importance features...


Extracting top features: 100%|██████████| 66/66 [00:00<00:00, 556.22it/s]


Step 2: Extracting rule-based features...


Extracting rule-based features: 100%|██████████| 66/66 [00:00<00:00, 253.16it/s]


Step 3: Extracting statistical features...
Step 4: Extracting embedding features...
Loading embedding model: intfloat/multilingual-e5-small
Loaded as SentenceTransformer model
Extracting embedding features for file_1...


...

Extracting embedding features for file_2...


Extracting embeddings: 100%|██████████| 13/13 [00:45<00:00,  3.48s/it]


Step 5: Extracting pairwise features...


Extracting pairwise features: 100%|██████████| 819/819 [00:01<00:00, 549.77it/s]

Step 6: Combining features...
Final feature matrix shape: (819, 1654)
Top features: 25, Rule: 78, Stat: 6, Embedding: 1538, Pairwise: 7
✅ Dự đoán cho các mẫu 'khó' hoàn tất.

--- BƯỚC 5: Tạo file submission ---
✅ Đã tạo file submission.csv thành công!
   id  real_text_id
0   0             2
1   1             2
2   2             1
3   3             1
4   4             2
